In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import euclidean_distances
from scipy import optimize

from scipy.io import savemat


with open('YALE_ABDOMINAL_PAIN.npy', 'rb') as f:
    X_train = np.load(f)
    y_train = np.load(f).reshape(-1)
    X_test = np.load(f)
    y_test = np.load(f).reshape(-1)

classes = ['0', '1']

n = X_train.shape[0]
    
print(X_train.shape, y_train.shape, X_train.dtype)
print(X_test.shape, y_test.shape, X_test.dtype)
    
%matplotlib notebook

%load_ext autoreload
%autoreload 2

In [ ]:
import umap

ump = umap.UMAP(n_neighbors=30,
        min_dist=0.1,
        n_components=2,
        random_state=150,
        metric= 'euclidean')

y_umap = ump.fit_transform(X_train)

plt.figure()
plt.scatter(y_umap[:,0], y_umap[:,1], c=y_train, s=0.1, cmap='Spectral')

cbar = plt.colorbar(boundaries=np.arange(11)-0.5)
cbar.set_ticks(np.arange(10))
cbar.set_ticklabels(classes)

In [ ]:
plt.figure()
plt.scatter(y_umap[:,0], y_umap[:,1], c=y_train, s=0.1, cmap='Spectral')

cbar = plt.colorbar(boundaries=np.arange(len(classes)+1)-0.5)
cbar.set_ticks(np.arange(len(classes)))
cbar.set_ticklabels(classes)

In [ ]:
dist = euclidean_distances(X_train, squared = False)

#with open('mnist_distances.npy', 'rb') as f:
#    dist = np.load(f)


#print(dist[0:4, 0:4])
print('\n')

    


In [ ]:
print(dist[0:4, 0:4])
print( np.sum((X_train[1,:] - (X_train[6,:]+X_train[5,:]))**2))

In [ ]:
n_neighbors=30

sort_idx = np.argsort(dist,axis=1)

sort_idx = sort_idx[:,1:n_neighbors+1]
print(sort_idx.shape, sort_idx[:,1].shape)


rho = [ dist[i, sort_idx[i,0] ] for i in range(n)]
rho = np.array(rho)

print(rho[0:4])
print(rho.shape, rho.dtype)

In [ ]:
#print(dist[0,[0,1,5,6]])
#print(dist[0,0:7])

import gc

print(gc.collect())

#with open('rho_mnist.npy', 'wb') as f:
#    np.save(f, rho)
    
#print(rho.shape)

In [ ]:
'''
def prob_high_dim(sigma, dist_row):
    """
    For each row of Euclidean distance matrix (dist_row) compute
    probability in high dimension (1D array)
    """
    d = dist[dist_row] - rho[dist_row];
    d[ d < 0 ] = 0
    return np.exp(-d / sigma)

def k(prob):
    """
    Computer n_neighbor = k (scalar) for each 1D array of high-dimensional probability
    """
    return np.power(2, np.sum(prob))
'''
temp = 0

In [ ]:
def get_weight_function(dists, rho, sigma):
    d = dists - rho
    #print(d)
    d[d<0] = 0
    weight = np.exp(- d / sigma )
    return weight


def search_sigma(dists, rho, k, tol = 10**-5, n_iteration=200):
    sigma_min = 0
    sigma_max = 1000
    
    cur_sigma = 100
    
    logk = np.log2(k)
    #print(logk)
    
    for i in range(n_iteration):
        
        cur_sigma = (sigma_min+sigma_max)/2
        probs = get_weight_function(dists,rho,cur_sigma)
        weight = np.sum(probs)
        #print(weight)
        
        if np.abs(logk - weight) < tol:
            break
        
        if weight < logk:
            sigma_min = cur_sigma
        else:
            sigma_max = cur_sigma
        
    return cur_sigma, probs

#sigma, weights = search_sigma(dists = dist[0,sort_idx[0,:]],rho = rho[0],k = n_neighbors)

#print(np.sum(np.exp( -(dist[0,1:] - rho[0]) / sigma ) ))

#print(dist[0,:] - rho[0], dist[0,0], rho[0])

#print(sigma)

In [ ]:
sigmas = []

directed_graph = []
#'''
for i in range(n):
    if (i+1)%10000 == 0:
        print('Processed ', i+1, ' of ', n, ' samples.')
    sigma, weights = search_sigma(dists = dist[i,sort_idx[i,:]],rho = rho[i],k = n_neighbors)
    
    probs = np.zeros(n)
    probs[sort_idx[i,:]] = weights
    #print(sum(weights), np.log2(n_neighbors))
    #print(sort_idx[i,:])
    #print(probs[1770:1780])
    
    directed_graph.append(probs)

prob = np.array(directed_graph)
#'''  

#with open('probs_n_neighbor30.npy', 'rb') as f:
#    prob = np.load(f)
    

#print(prob.shape)


#print(prob[0:10,0:10])

In [ ]:
import gc
gc.collect()

#'''
import numba 
from numba import prange
@numba.jit(nopython=True, parallel=True)
def symmetrization_step(prob):
    P = np.zeros((n,n),dtype=np.float32)

    for i in prange(n):
        #if i%1000 == 0:
        #    print('Completed ', i, ' of ', n)
        for j in prange(i,n):
            p = prob[i,j] + prob[j,i] - prob[i,j] * prob[j,i]
            P[i,j] = p
            P[j,i] = p
            
    return P


P = symmetrization_step(prob)
#'''
print(np.sum(P[0,:]==0))

    

print(P.shape)

In [ ]:
print(P[0,P[0,:]>0])
np.sum(P[0,P[0,:]>0])

In [ ]:
MIN_DIST = 0.1

x = np.linspace(0, 3, 300)
y = np.exp(- (x-MIN_DIST) * ( (x - MIN_DIST) >=0 ) )

dist_low_dim = lambda x, a, b: 1 / (1 + a*x**(2*b))

p , _ = optimize.curve_fit(dist_low_dim, x, y)

a = p[0]
b = p[1] 
print("Hyperparameters a = " + str(a) + " and b = " + str(b))

x_p = np.arange(0,3,0.01)
y_p = np.exp(- (x_p-MIN_DIST) * ( (x_p - MIN_DIST) >=0 ) )
y_p2 = 1 / (1 + a*x_p**(2*b))

plt.figure()
plt.plot(x,y, label='Target')
plt.plot(x_p,y_p2, label='Fitted')

In [ ]:
import torch
from network_sig import network

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = network(channels=[X_train.shape[1],500,300,200,100,100,100,2]).to(device)


criterion_mse = torch.nn.MSELoss()

lr = 0.001

optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999))


In [ ]:
Y = []
X_train32 = X_train.astype(np.float32)

print('Conversion Done')
X_torch = torch.as_tensor(X_train32, dtype=torch.float32).to(device)
Y_umap = torch.as_tensor(y_umap, dtype=torch.float32).to(device)

X_torch_test = torch.as_tensor(X_test, dtype=torch.float32).to(device)
print('Tensor Build Done')


In [ ]:
Y = []

with torch.no_grad():
    for i in range(n):
        if i%10000 == 0:
            print('completed ', i, ' of ', n)
            
        x_st = X_torch[i:i+1,:]
        #print(x_st.size())
        y_st = model(x_st).cpu().numpy().reshape(-1)
        #print(y_st.shape)
        Y.append(y_st)

Y = np.array(Y)
print(Y.shape)


plt.figure()#figsize=(20,15))
plt.scatter(Y[:,0], Y[:,1], c = y_train.astype(int), cmap = 'Spectral', s = 20)
plt.title("Random Init")
plt.xlabel("Axis-1") #, fontsize = 20); 
plt.ylabel("Axis-2") #, fontsize = 20)
cbar = plt.colorbar(boundaries=np.arange(11)-0.5)
cbar.set_ticks(np.arange(10))
cbar.set_ticklabels(classes)
plt.show()
plt.show()

In [ ]:
def criterion_pos_old(x,y,a,b):
    factor = 1 + a * torch.sum((x-y)**2, dim = 0) ** b
    y = torch.log(factor)
    
    return y

def criterion_neg_old(x,y,a,b):
    factor = a * torch.sum((x-y)**2) ** b
    y = - torch.log(0.0001+factor) + torch.log(1 + factor)
    
    return y

def phi(x,y,a,b):
    factor = 1 + a * torch.sum((x-y)**2+10**-8, dim=1) ** b
    y = 1/factor
    
    #for i in range(y.size()[0]):
    #    if np.isnan(y[i].cpu().detach().item()):
    #        print('Isnan in Pos Crit:', i, x[i], y[i])
    
    return y

def criterion_pos(x,y,p,a,b):
    prob = phi(x,y,a,b)
    y = -torch.log(prob)*p
    #for i in range(y.size()[0]):
    #    if np.isnan(y[i].cpu().detach().item()):
    #        print('Isnan in Pos Crit:', i)
    
    return torch.sum(y)

def criterion_neg(x,y,p,a,b):
    prob = phi(x,y,a,b)
    y = -torch.log(1-prob+0.000001)*(1-p)
    #for i in range(y.size()[0]):
    #    if np.isnan(y[i].cpu().detach().item()):
    #        print('Isnan in Neg Crit:', i)
    
    return torch.sum(y)
    

Y = model(X_torch[0:10,:])
Z = criterion_pos(Y[0:4,:],Y[4:8,:],1.0,1.0,1.0)
print(Y.size(), Z.size(), Z.item())

In [ ]:
epochs = 40

print(n)

batch_size = 60
it_per_epoch = int(n/batch_size)
print('Iteration Per Epoch:', it_per_epoch)

CE_array = []
results = []
print("Running Gradient Descent: \n")

Nan_info = False

for epoch in range(epochs):
    
    for idx in range(n): #:
        choices_0 = np.random.choice(n, batch_size)
        k_ch = np.random.randint(low = 0, high = n_neighbors, size=batch_size)
        choices_1 = sort_idx[choices_0,k_ch]
        
        #print(choices_0.shape,choices_1.shape)
        
        Prob_sample = torch.as_tensor(P[choices_0,choices_1].reshape(-1), dtype=torch.float32).to(device)
        
        optimizer.zero_grad()
        
        loss = 0
        
        Y0 = model(X_torch[choices_0,:])
        Y1 = model(X_torch[choices_1,:])
        
        cr = criterion_pos(Y0, Y1, Prob_sample, a, b)
        loss = loss + cr
        #print(cr)
        
        
        if np.isnan(cr.detach().cpu().item()):
            Nan_info = True
            print(epoch,idx,cr)
            break
        
        CE_array.append(cr.cpu().detach().item())
    
        
        for j in range(5):
            k_ch = np.random.randint(low = 0, high = n, size=batch_size)
            Y1 = model(X_torch[k_ch,:])
            Prob_sample = torch.as_tensor(P[choices_0,k_ch].reshape(-1), dtype=torch.float32).to(device)
            
            cr = criterion_neg(Y0, Y1, Prob_sample, a, b)
            loss = loss + cr
            if np.isnan(cr.detach().cpu().item()):
                Nan_info = True
                print('inside loop: ', epoch, idx, j, cr)
                break
            
        if Nan_info == True:
            print('EVERYTHING IS NAN')
            break
        loss.backward()
        optimizer.step()
        
                
    #LEARNING_RATE = 1.0 - epoch / epochs
    if (epoch+1)%1 == 0:
        print('Completed ', epoch , ' of ', epochs)
        torch.save(model.state_dict(), 'nets_CE/epoch'+str(epoch)+'.pth')
        
        Y = []
        Y_test = []
        with torch.no_grad():
            for i in range(n):
                x_st = X_torch[i:i+1,:]
                y_st = model(x_st).cpu().numpy().reshape(-1)
                #print(y_st.shape)
                Y.append(y_st)
            

            
            for i in range(X_test.shape[0]):
                x_st = X_torch_test[i:i+1,:]
                #print(x_st.size())
                y_st = model(x_st).cpu().numpy().reshape(-1)
                #print(y_st.shape)
                Y_test.append(y_st)
        
        Y_test = np.array(Y_test)
        Y = np.array(Y)
        
        neigh.fit(Y, y_train) 
        y_nene_out = neigh.predict(Y_test)
        
        result = 1-np.mean(y_nene_out==y_test)
        
        results.append(result)
        
        d = {}
        d['Y'] = Y
        d['Y_test'] = Y_test
        savemat('nets_CE/test_data'+str(epoch)+'.mat', d)
        
        print('Error :', result)
        
    if (epoch+1)/5 == 0:
        lr = lr/10
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

In [ ]:
Y = []

with torch.no_grad():
    for i in range(n):
        if i%10000 == 0:
            print('completed ', i, ' of ', n)
            
        x_st = X_torch[i:i+1,:]
        #print(x_st.size())
        y_st = model(x_st).cpu().numpy().reshape(-1)
        #print(y_st.shape)
        Y.append(y_st)

Y = np.array(Y)
print(Y)

plt.figure()#figsize=(20,15))
plt.scatter(Y[:,0], Y[:,1], c = y_train.astype(int), cmap = 'Spectral', s = 20)
plt.title("UMAP")
plt.xlabel("UMAP 1") #, fontsize = 20); 
plt.ylabel("UMAP 2") #, fontsize = 20)
cbar = plt.colorbar(boundaries=np.arange(11)-0.5)
cbar.set_ticks(np.arange(10))
cbar.set_ticklabels(classes)
plt.show()
plt.show()